In [11]:

import os
import copy
import time
import itertools as it

import networkx as nx
import numpy as np


# Implementation of Basic function

Here is some basic function to tranform SAT file to weighted_adjacency matrix and get clique from graph.

In [12]:
def read_sat(sat_path):
    with open(sat_path) as f:
        sat_lines = f.readlines()
        header = sat_lines[0]
        header_info = header.replace("\n", "").split(" ")
        vars_num = int(header_info[-2])
        clauses_num = int(header_info[-1])

        sat = [
            [int(x) for x in line.replace(" 0\n", "").split(" ")]
            for line in sat_lines[1:]
        ]

        return vars_num, clauses_num, sat


def sat_to_lig_adjacency_matrix(sat, num_vars):
    def get_literal_idx(x): return 2 * x - 2 if x > 0 else 2 * abs(x) - 1
    lig_adjacency_matrix = np.zeros([2*num_vars, 2*num_vars])
    lig_weighted_adjacency_matrix = np.zeros([2*num_vars, 2*num_vars])

    for clause in sat:
        pairs = it.combinations(clause, 2)
        for x, y in pairs:
            x_idx = get_literal_idx(x)
            y_idx = get_literal_idx(y)
            lig_adjacency_matrix[x_idx, y_idx] = 1
            lig_adjacency_matrix[y_idx, x_idx] = 1
            lig_weighted_adjacency_matrix[x_idx, y_idx] += 1
            lig_weighted_adjacency_matrix[y_idx, x_idx] += 1
    return lig_weighted_adjacency_matrix


def get_cliques(graph, k):
    cliques = nx.enumerate_all_cliques(graph)
    clique_candidates = []
    for clique in cliques:
        if len(clique) <= k:
            if len(clique) > 1:
                clique_candidates.append(clique)
        else:
            break
    return clique_candidates


def cliques_to_weighted_adjacency_matrix(cliques, num_vars):
    weighted_adjacency_matrix = np.zeros([2*num_vars, 2*num_vars])
    for clique in cliques:
        pairs = it.combinations(clique, 2)
        for pair in pairs:
            x_idx = pair[0]
            y_idx = pair[1]

            weighted_adjacency_matrix[x_idx, y_idx] += 1
            weighted_adjacency_matrix[y_idx, x_idx] += 1

    return weighted_adjacency_matrix


def objective(x, y):
    return np.abs(x - y).sum()/np.count_nonzero(x)




# Convenience function: `run`

As like TSP notebook, I will define `run` to run a clique cover operator (algorithm) on given: weighted adjacency matrix, valid_cliques, and select_clique_num.

In [13]:
def run(cover_opt, instance, weighted_adjacency_matrix, max_clique_size, select_clique_num, num_vars):
    print(f"{'<' * 8} {instance} {'>' * 8}")
    # get the valid cliques from weighted_adjacency_matrix (graph)
    start_time = time.time()
    graph = nx.from_numpy_matrix(weighted_adjacency_matrix)
    valid_cliques  = get_cliques(graph, max_clique_size)
    print(f'cliques enumerate time: {time.time() - start_time:.4f}')
    print(f'valid cliques num: {len(valid_cliques)}')

    # select cliques from valid cliques from the valid cliques
    # your cover_opt will be used in here
    start_time = time.time()
    select_cliques = cover_opt(copy.deepcopy(weighted_adjacency_matrix), valid_cliques, select_clique_num)
    print(f'cliques cover time: {time.time() - start_time:.4f}')

    # convert the selected cliques to weighted matrix and compare to the given weigthed matrix
    weighted_adjacency_matrix_cover = cliques_to_weighted_adjacency_matrix(select_cliques, num_vars)
    print(f'final objective: {objective(weighted_adjacency_matrix, weighted_adjacency_matrix_cover):.4f}\n') 

# Implement and evaluate your algorithm

Your algortihm will be evaluate on three graphs (sat instances).

In [14]:
# write you code in here

def naive_clique_cover(weighted_adjacency_matrix, valid_cliques, select_clique_num):
    return valid_cliques[:select_clique_num]

# the below code will evaluate your algorithm

instances = ['ssa2670-141.processed.cnf', 'mrpp_4x4#4_5.processed.cnf', 'bmc-ibm-7.processed.cnf']
for instance in instances:
    sat_path = os.path.join('./formulas', instance)
    num_vars, num_clauses, sat_instance = read_sat(sat_path)
    weighted_adjacency_matrix = sat_to_lig_adjacency_matrix(sat_instance, num_vars)
    max_clique_size = max([len(clause) for clause in sat_instance])
    
    run(naive_clique_cover, instance, weighted_adjacency_matrix, max_clique_size, num_clauses, num_vars)



<<<<<<<< ssa2670-141.processed.cnf >>>>>>>>
cliques enumerate time: 0.0080
valid cliques num: 4822
cliques cover time: 0.0001
final objective: 1.0716

<<<<<<<< mrpp_4x4#4_5.processed.cnf >>>>>>>>
cliques enumerate time: 0.1602
valid cliques num: 80390
cliques cover time: 0.0001
final objective: 1.3907

<<<<<<<< bmc-ibm-7.processed.cnf >>>>>>>>
cliques enumerate time: 10.2834
valid cliques num: 1805257
cliques cover time: 0.0047
final objective: 1.4059

